In [2]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2022-11-09 17:06:39--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 2607:f8b0:4023:c0b::80, 2607:f8b0:4023:c0d::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.008s  

2022-11-09 17:06:39 (312 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [3]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []

for i, line in enumerate(lines):
    print(line)
    if i==20:
        break

for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

print(english)
print(spanish)

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!
Fire!	¡Disparad!
Help!	¡Ayuda!
Help!	¡Socorro! ¡Auxilio!
Help!	¡Auxilio!
Jump!	¡Salta!
Jump.	Salte.
Stop!	¡Parad!
Stop!	¡Para!
Stop!	¡Pare!
Wait!	¡Espera!
Wait.	Esperen.
If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.
[start] Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado. [end]


In [4]:
print(text_pairs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
import random
print(random.choice(text_pairs))

('The girl has a soft heart.', '[start] La niña tiene un corazón blando. [end]')


In [6]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [7]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
print(strip_chars)

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

!"#$%&'()*+,-./:;<=>?@\^_`{|}~¿


In [8]:
print(f"[{re.escape(strip_chars)}]")

[!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\\\^_`\{\|\}\~¿]


In [9]:
print(re.escape(strip_chars))

!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\\\^_`\{\|\}\~¿


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [11]:
for pair in train_pairs:
    print(pair[0])
    print(pair[1])
    break

I think you panicked.
[start] Creo que te entró miedo. [end]


In [12]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    for i,x in enumerate(eng_texts):
        #print(i,x)
        if i==68:
            break
    
    for i,x in enumerate(spa_texts):
        #print(i,x)
        if i==68:
            break
    eng_texts = list(eng_texts)

    #print(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices(eng_texts)
    
    for i,ele in enumerate(dataset):
        print(i, ele)
        if i==1:
            break
    for el in dataset:
        print(el.numpy())
        break

    #print(dataset)
    dataset = dataset.batch(batch_size)

    for i,el in enumerate(dataset):
        print(i, el)
        if i==0:
            break

    dataset = dataset.map(format_dataset, num_parallel_calls=4)

    
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

0 tf.Tensor(b'I think you panicked.', shape=(), dtype=string)
1 tf.Tensor(b'How can I prevent my cat from killing birds?', shape=(), dtype=string)
b'I think you panicked.'
0 tf.Tensor(
[b'I think you panicked.' b'How can I prevent my cat from killing birds?'
 b'Nothing will help.' b'Tom drank too much.' b'Mary is glowing.'
 b'The French like to make fun of Belgians.'
 b'Because of the poor harvest, wheat prices have gone up in the last six months.'
 b"Tom's father was an accountant."
 b'The pitiful sight moved us to tears.'
 b'No matter what happens, Tom always shows up on time.'
 b"The accident damaged her car's front wheels."
 b"Tom didn't seem very enthusiastic." b"We don't have that long."
 b'Use this as a model.' b"What do you think you're doing?"
 b"It's not logical." b'It looks like Tom likes his new job.'
 b"I don't know whether he is dead or alive."
 b"What's your native language?" b'I had to go to Boston.'
 b'She cheered me up when I was feeling blue.'
 b'Tom tried to hide be

TypeError: ignored

In [13]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [14]:
for inputs, targets in train_ds.take(1):         #it essentially "takes" that many elements from the dataset. I'm guessing that in this specific example, 
                                                #someone wanted to show what the data looked like and took only one element. 
    print(inputs)                                            #If you don't use take, all elements will eventually be fetched 
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

{'english': <tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[  15,    5,   32, ...,    0,    0,    0],
       [ 112,  194,   57, ...,    0,    0,    0],
       [  81,  152, 1047, ...,    0,    0,    0],
       ...,
       [ 188,    8,    7, ...,    0,    0,    0],
       [  48,    8,  191, ...,    0,    0,    0],
       [  23,    5,   10, ...,    0,    0,    0]])>, 'spanish': <tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[   2,  124, 1833, ...,    0,    0,    0],
       [   2,   77,  302, ...,    0,    0,    0],
       [   2,  147,   83, ...,    0,    0,    0],
       ...,
       [   2,   10,  156, ...,    0,    0,    0],
       [   2,   89,  116, ...,    0,    0,    0],
       [   2,   83,    4, ...,    0,    0,    0]])>}
inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


Seq to seq with RNNs

In [15]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

In [16]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [17]:

seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
  10/1302 [..............................] - ETA: 1:58:01 - loss: 3.0072 - accuracy: 0.1357

KeyboardInterrupt: ignored

In [18]:
x= [1,3,4,5,6,64,3,6,5,63]
len(x)

10

In [19]:
y=dict(zip(range(len(x)),x))
y

{0: 1, 1: 3, 2: 4, 3: 5, 4: 6, 5: 64, 6: 3, 7: 6, 8: 5, 9: 63}

In [21]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Nothing will happen until 2:30.
1/1 [==============================] - 0s 120ms/step
[start] [end]
-
Speaking English is very difficult for me.
1/1 [==============================] - 0s 125ms/step
[start] [end]
-
I can't believe he did this to me.
1/1 [==============================] - 0s 128ms/step
[start] a [end]
-
Tom is your guitar teacher, isn't he?
1/1 [==============================] - 0s 129ms/step
[start] a [end]
-
You lied to me, didn't you?
1/1 [==============================] - 0s 114ms/step
[start] [end]
-
Her attitude disgusts me.
1/1 [==============================] - 0s 124ms/step
[start] [end]
-
The policeman said that it looked like a self-inflicted gunshot wound to the head.
1/1 [==============================] - 0s 120ms/step
[start] a [end]
-
She looks sad.
1/1 [==============================] - 0s 116ms/step
[start] [end]
-
I don't feel like studying at all tonight.
1/1 [==============================] - 0s 115ms/step
[start] [end]
-
I think that I drank too muc